<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/WOOK/%EC%95%99%EC%83%81%EB%B8%94(%EB%B3%B4%ED%8C%85).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Labels_Safe_MSR+Juliet_no_random.csv의 사본')

In [17]:
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder
from datasets import load_dataset
df = df.drop(columns=["vul", "Unnamed: 0"])
df['code'] = df['code'].apply(lambda x: re.sub(r'/\*.*?\*/', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'//.*?\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#include <.*?>\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#include \".*?\"\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#define .*?\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'[\t ]+', ' ', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'\n\s*\n', '\n', x))
df.to_csv("preprocessed_data.csv", index=False)

dataset = load_dataset("csv", data_files="preprocessed_data.csv")['train']

Generating train split: 0 examples [00:00, ? examples/s]

In [22]:
df.head(10)

,CWE ID,lang,code,labels
0,safe,c,"void func(fz_context *ctx, fz_color_converter ...",0
1,safe,c,"void func(fz_context *ctx, fz_color_converter ...",0
2,safe,c,"void func(fz_context *ctx, const fz_colorspace...",0
3,safe,c,static inline void func(unsigned char *restric...,0
4,safe,c,"void func(const fz_colorspace *cs, const float...",0
5,safe,c,"void func(const fz_colorspace *cs, const float...",0
6,safe,c,"void func(const fz_colorspace *cs, const float...",0
7,safe,c,"void func(const fz_colorspace *cs, const float...",0
8,safe,c,"void func(const fz_colorspace *cs, const float...",0
9,safe,c,"void func(fz_context *ctx, fz_color_converter ...",0


In [6]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from sklearn.model_selection import KFold
import torch
from torch.utils.data import DataLoader

In [23]:
# 데이터셋 로딩 및 토큰화 함수
def load_and_tokenize_data(filepath, tokenizer):
    dataset = load_dataset("csv", data_files=filepath)['train']
    tokenized_dataset = dataset.map(lambda x: tokenizer(x['code'], padding="max_length", truncation=True, max_length=512), batched=True)
    return tokenized_dataset

In [24]:
# 모델 및 토크나이저 설정
model_names = [
    "microsoft/graphcodebert-base",
    "microsoft/codebert-base-mlm",
    "microsoft/unixcoder-base"
]
models = [AutoModelForSequenceClassification.from_pretrained(name, num_labels=11) for name in model_names]
tokenizers = [AutoTokenizer.from_pretrained(name) for name in model_names]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base-mlm and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/unixcoder-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out

In [25]:
# K-Fold 설정 및 훈련
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
data_filepath = '/content/preprocessed_data.csv'
trained_models = []



In [27]:
from transformers import TrainerCallback, TrainerControl, TrainerState
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

class F1ScoreCallback(TrainerCallback):
    def __init__(self, num_labels):
        super().__init__()
        self.label_names = [f'Label {i}' for i in range(num_labels)]  # 라벨 이름을 'Label 0', 'Label 1', ..., 'Label 10'로 설정

    def on_evaluate(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        if state.is_local_process_zero:
            eval_dataloader = kwargs.get("eval_dataloader")
            model = kwargs.get("model")
            device = model.device

            predictions = []
            labels = []

            for batch in eval_dataloader:
                batch = {k: v.to(device) for k, v in batch.items()}
                with torch.no_grad():
                    outputs = model(**batch)

                logits = outputs.logits
                preds = torch.argmax(logits, dim=-1)
                predictions.extend(preds.cpu().numpy())
                labels.extend(batch['labels'].cpu().numpy())

            # 라벨별로 F1, 정밀도, 재현율 계산
            f1_scores = f1_score(labels, predictions, average=None)
            precision_scores = precision_score(labels, predictions, average=None)
            recall_scores = recall_score(labels, predictions, average=None)

            # 라벨별 성능 지표 출력
            print("Label-wise Metrics:")
            for i, (f1, precision, recall) in enumerate(zip(f1_scores, precision_scores, recall_scores)):
                print(f"{self.label_names[i]}: F1={f1:.4f}, Precision={precision:.4f}, Recall={recall:.4f}")



In [28]:
for model, tokenizer in zip(models, tokenizers):
    tokenized_dataset = load_and_tokenize_data(data_filepath, tokenizer)
    for fold, (train_idx, val_idx) in enumerate(kf.split(tokenized_dataset)):
        train_subset = tokenized_dataset.select(train_idx)
        val_subset = tokenized_dataset.select(val_idx)
        training_args = TrainingArguments(
            output_dir=f'./results/fold_{fold}',
            evaluation_strategy="epoch",
            learning_rate=2e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=64,
            num_train_epochs=1,
            weight_decay=0.01
        )
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_subset,
            eval_dataset=val_subset,
            callbacks=[F1ScoreCallback(num_labels=11)]  # 11개 라벨 설정
        )

        trainer.train()
        trained_models.append((model, tokenizer, val_subset))

Map:   0%|          | 0/16401 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,1.086300,0.709644


Label-wise Metrics:
Label 0: F1=0.8000, Precision=0.7792, Recall=0.8220
Label 1: F1=0.5876, Precision=0.4669, Recall=0.7925
Label 2: F1=0.9195, Precision=0.8549, Recall=0.9947
Label 3: F1=0.8689, Precision=0.8824, Recall=0.8559
Label 4: F1=0.8764, Precision=0.8619, Recall=0.8914
Label 5: F1=0.9326, Precision=0.9574, Recall=0.9091
Label 6: F1=0.4367, Precision=0.4505, Recall=0.4237
Label 7: F1=0.8235, Precision=0.9385, Recall=0.7336
Label 8: F1=0.6943, Precision=1.0000, Recall=0.5318
Label 9: F1=0.1514, Precision=0.5000, Recall=0.0892
Label 10: F1=0.4242, Precision=0.8974, Recall=0.2778


Epoch,Training Loss,Validation Loss
1,0.658000,0.519890


Label-wise Metrics:
Label 0: F1=0.8866, Precision=0.8897, Recall=0.8835
Label 1: F1=0.6879, Precision=0.6158, Recall=0.7792
Label 2: F1=0.9344, Precision=0.8832, Recall=0.9918
Label 3: F1=0.8994, Precision=0.8750, Recall=0.9253
Label 4: F1=0.9061, Precision=0.9257, Recall=0.8872
Label 5: F1=0.9482, Precision=0.9734, Recall=0.9243
Label 6: F1=0.4775, Precision=0.5233, Recall=0.4391
Label 7: F1=0.9116, Precision=0.9469, Recall=0.8789
Label 8: F1=0.7445, Precision=1.0000, Recall=0.5930
Label 9: F1=0.4922, Precision=0.4737, Recall=0.5122
Label 10: F1=0.5803, Precision=0.7467, Recall=0.4746


...

Label-wise Metrics:
Label 0: F1=0.9222, Precision=0.9371, Recall=0.9078
Label 1: F1=0.7740, Precision=0.6798, Recall=0.8985
Label 2: F1=0.9398, Precision=0.8865, Recall=1.0000
Label 3: F1=0.9032, Precision=0.9423, Recall=0.8673
Label 4: F1=0.9026, Precision=0.8479, Recall=0.9648
Label 5: F1=0.9383, Precision=0.9720, Recall=0.9070
Label 6: F1=0.6069, Precision=0.7372, Recall=0.5157
Label 7: F1=0.9255, Precision=0.9361, Recall=0.9152
Label 8: F1=0.8188, Precision=0.9037, Recall=0.7485
Label 9: F1=0.6551, Precision=0.6963, Recall=0.6184
Label 10: F1=0.7547, Precision=0.8889, Recall=0.6557


Epoch,Training Loss,Validation Loss
1,0.409200,0.330914


Label-wise Metrics:
Label 0: F1=0.9255, Precision=0.9491, Recall=0.9031
Label 1: F1=0.8307, Precision=0.7820, Recall=0.8860
Label 2: F1=0.9254, Precision=0.8655, Recall=0.9942
Label 3: F1=0.8932, Precision=0.8895, Recall=0.8969
Label 4: F1=0.8997, Precision=0.8971, Recall=0.9023
Label 5: F1=0.9596, Precision=0.9727, Recall=0.9468
Label 6: F1=0.7645, Precision=0.7872, Recall=0.7430
Label 7: F1=0.9438, Precision=0.9633, Recall=0.9251
Label 8: F1=0.8780, Precision=0.9536, Recall=0.8136
Label 9: F1=0.7619, Precision=0.8000, Recall=0.7273
Label 10: F1=0.7725, Precision=0.8182, Recall=0.7317


Epoch,Training Loss,Validation Loss
1,0.377200,0.266943


Label-wise Metrics:
Label 0: F1=0.9247, Precision=0.9270, Recall=0.9224
Label 1: F1=0.8688, Precision=0.8277, Recall=0.9142
Label 2: F1=0.9246, Precision=0.8659, Recall=0.9919
Label 3: F1=0.8794, Precision=0.9730, Recall=0.8022
Label 4: F1=0.8979, Precision=0.8492, Recall=0.9525
Label 5: F1=0.9556, Precision=0.9929, Recall=0.9211
Label 6: F1=0.7970, Precision=0.8154, Recall=0.7794
Label 7: F1=0.9583, Precision=0.9673, Recall=0.9495
Label 8: F1=0.9103, Precision=0.9595, Recall=0.8659
Label 9: F1=0.7857, Precision=0.8462, Recall=0.7333
Label 10: F1=0.8625, Precision=0.8722, Recall=0.8529


In [32]:
import torch
import numpy as np
from sklearn.metrics import f1_score

# GPU 사용 가능 여부에 따라 device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def soft_voting(predictions):
    # 각 모델의 예측 확률을 평균내어 최종 예측을 결정
    ensemble_probs = torch.mean(torch.stack(predictions, dim=0), dim=0)
    final_predictions = torch.argmax(ensemble_probs, dim=-1)
    return final_predictions.cpu().numpy()


def evaluate_and_collect_predictions(models_tokenizers_datasets):
    all_predictions = []
    all_labels = []
    for model, tokenizer, dataset in models_tokenizers_datasets:
        model.eval()
        model.to(device)
        predictions = []
        labels = []

        # 데이터셋을 DataLoader를 사용하여 반복 처리
        data_loader = DataLoader(dataset, batch_size=64, shuffle=False)
        for batch in data_loader:
            inputs = tokenizer(batch['code'], return_tensors="pt", padding="max_length", truncation=True, max_length=512)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predictions.append(probs)
            labels.extend(batch['labels'])  # 레이블 수집

        # 모든 배치에 대한 예측을 취합
        predictions = torch.cat(predictions, dim=0)
        all_predictions.append(predictions)
        all_labels.append(torch.tensor(labels, dtype=torch.long))  # 레이블을 텐서로 변환

    return all_predictions, all_labels


# 이 함수를 사용하여 예측과 레이블을 수집하세요
predictions, labels = evaluate_and_collect_predictions(trained_models)

# 소프트 보팅 적용
final_predictions = soft_voting(predictions)

# 소프트 보팅 결과를 사용하여 F1 점수 계산
final_f1_score = f1_score(labels[0].numpy(), final_predictions, average='macro')
print(f"소프트 보팅 후 최종 F1 매크로 점수: {final_f1_score}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 72.81 MiB is free. Process 3216 has 39.48 GiB memory in use. Of the allocated memory 38.65 GiB is allocated by PyTorch, and 342.80 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [35]:
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/model/voting model1.pth')